# Analysis of Powerlifting Meets

## Elias Prieto, UID: 116643344, May 12th, 2023

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

## Part 1 - Retrieving the Data

In [2]:
# reading CSV through pandas
df = pd.read_csv('powerlifting.csv', dtype=object)

# displaying head of Dataframe
df.head()

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
0,Alona Vladi,F,SBD,Raw,33,24-34,24-39,O,58.3,60,...,Yes,Russia,NaN,GFP,NaN,2019-05-11,Russia,NaN,Bryansk,Open Tournament
1,Galina Solovyanova,F,SBD,Raw,43,40-44,40-49,M1,73.1,75,...,Yes,Russia,NaN,GFP,NaN,2019-05-11,Russia,NaN,Bryansk,Open Tournament
2,Daniil Voronin,M,SBD,Raw,15.5,16-17,14-18,T,67.4,75,...,Yes,Russia,NaN,GFP,NaN,2019-05-11,Russia,NaN,Bryansk,Open Tournament
3,Aleksey Krasov,M,SBD,Raw,35,35-39,24-39,O,66.65,75,...,Yes,Russia,NaN,GFP,NaN,2019-05-11,Russia,NaN,Bryansk,Open Tournament
4,Margarita Pleschenkova,M,SBD,Raw,26.5,24-34,24-39,O,72.45,75,...,Yes,Russia,NaN,GFP,NaN,2019-05-11,Russia,NaN,Bryansk,Open Tournament


In [ ]:
df.columns

## Part 2 - Tidying Data

Factors to Consider:
- Only want to analyze lifts in USAPL and IPF as they are drug tested and judges are strict on the validity of a given lift
- Need to seperate men and women as they normally compete seperately 
- Need to make a standardized metric for age groups, weights and weight classes, etc
- Only consider data from SBD competitions since single lift competitions can skew results (maybe give a chart showing this)
- Take out missing data and give explanation as to why it might be missing and why we don't want to include it
- Some data is negative for lifts, etc, need to filter this out

In [3]:
# get rid of unnecessary columns
df = df[['Date', 'Age', 'Sex', 'Event', 'BirthYearClass', 'BodyweightKg', \
       'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Best3SquatKg', \
       'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Best3BenchKg', \
       'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', \
       'Best3DeadliftKg', 'TotalKg', 'Place', 'Dots', 'Wilks', 'Glossbrenner', \
       'Tested', 'Federation']]


# drop all rows with Nan values since we only want to analyze people who completed all their lifts
# and have important data like their weight and age/birthyearclass
df = df.dropna()

# restrict to just USAPL and IPF federations
df = df[(df['Federation'] == 'USAPL') | (df['Federation'] == 'IPF')]

# restrict to SBD and Tested meets only
df = df[(df['Event'] == 'SBD') & (df['Tested'] == 'Yes')]
df.head()

,Date,Age,Sex,Event,BirthYearClass,BodyweightKg,Squat1Kg,Squat2Kg,Squat3Kg,Best3SquatKg,...,Deadlift2Kg,Deadlift3Kg,Best3DeadliftKg,TotalKg,Place,Dots,Wilks,Glossbrenner,Tested,Federation
307302,2018-11-10,48,F,SBD,40-49,51.35,102.5,110,112.5,112.5,...,122.5,127.5,127.5,292.5,1,359.67,368.19,326.53,Yes,USAPL
307304,2018-11-10,47,F,SBD,40-49,71.6,100,112.5,120,120,...,142.5,150,150,325,1,324.56,318.39,280.50,Yes,USAPL
307305,2018-11-10,43.5,F,SBD,40-49,114.65,70,80,92.5,92.5,...,127.5,140,140,315,1,256.19,254.00,215.78,Yes,USAPL
307306,2018-11-10,38.5,F,SBD,24-39,51.1,92.5,95,97.5,97.5,...,117.5,125,125,275,1,339.31,347.46,308.21,Yes,USAPL
307307,2018-11-10,26.5,F,SBD,24-39,57,92.5,97.5,102.5,102.5,...,112.5,115,115,277.5,1,317.93,322.01,284.73,Yes,USAPL


In [4]:
for i,row in df.iterrows():
    row['Date'] = pd.to_datetime(row['Date'])
    try:
        row['Age'] = float(row['Age'])
    except:
        row['Age'] = np.nan
    # want to nake distinction between Male and Female 
    if row['Sex'] == 'Mx':
        row['Sex'] = np.nan
        
    lifts = ['Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Best3SquatKg', \
       'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Best3BenchKg', \
       'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', \
       'Best3DeadliftKg', 'TotalKg']
      
    # cast all lift values as floats and get rid of invalid data
    for lift in lifts:
        try:
            row[lift] = float(row[lift])
            if row[lift] <= 0:
                 row[lift] = np.nan
        except:
             row[lift] = np.nan
    # cast all places as integers and get rid of invalid data
    try:
        row['Place'] = int(row['Place'])
        if row['Place'] > 0:
            row['Place'] = int(row['Place'])
        else:
            row['Place'] = np.nan
    except:
        row['Place'] = np.nan
    
    # cast all scores as floats and get rid of invalid data         
    for score in ['Dots', 'Wilks', 'Glossbrenner']:
        try:
            row[score] = float(row[score])  
            if row[score] > 0:
                 row[score] = float(row[score])
            else:
                row[score] = np.nan
        except:
            row[score] = np.nan
cleanedData = df.dropna()
# We fixed the event to be SBD and the Federations to only be USAPL and IPF which are tested federations
cleanedData = cleanedData.drop(['Event', 'Tested', 'Federation'], axis=1)
cleanedData.head()

,Date,Age,Sex,BirthYearClass,BodyweightKg,Squat1Kg,Squat2Kg,Squat3Kg,Best3SquatKg,Bench1Kg,...,Best3BenchKg,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Best3DeadliftKg,TotalKg,Place,Dots,Wilks,Glossbrenner
307305,2018-11-10 00:00:00,43.5,F,40-49,114.65,70.0,80.0,92.5,92.5,65.0,...,82.5,110.0,127.5,140.0,140.0,315.0,1,256.19,254.0,215.78
307309,2018-11-10 00:00:00,24.5,F,24-39,58.95,102.5,110.0,115.0,115.0,57.5,...,62.5,127.5,135.0,142.5,142.5,320.0,2,358.73,361.69,319.65
307314,2018-11-10 00:00:00,24.5,F,24-39,71,110.0,117.5,122.5,122.5,67.5,...,75.0,137.5,147.5,155.0,155.0,352.5,3,353.66,347.3,306.03
307318,2018-11-10 00:00:00,30.5,F,24-39,97.15,150.0,165.0,172.5,172.5,65.0,...,77.5,165.0,175.0,187.5,187.5,437.5,1,377.64,367.52,316.84
307319,2018-11-10 00:00:00,43.5,F,40-49,114.65,70.0,80.0,92.5,92.5,65.0,...,82.5,110.0,127.5,140.0,140.0,315.0,2,256.19,254.0,215.78


In [9]:
#cleanedData = cleanedData.astype({'Date': 'datetime64[ns]', 'Age': 'float64', 'BodyweightKg': 'float64', \
#                                 'Squat1Kg': 'float64', 'Squat2Kg': 'float64', 'Squat3Kg': 'float64', 'Best3SquatKg': 'float64',\
#                                 'Bench1Kg': 'float64', 'Bench2Kg': 'float64', 'Bench3Kg': 'float64', 'Best3BenchKg': 'float64',\
#                                 'Deadlift1Kg': 'float64', 'Deadlift2Kg': 'float64', 'Deadlift3Kg': 'float64', 'Best3DeadliftKg': 'float64',\
#                                 'TotalKg': 'float64', 'Place': 'int', 'Dots': 'float64', 'Wilks': 'float64',  'Glossbrenner': 'float64'})

#cleanedData.dtypes

# save cleaned data so that I can do analysis without having to run all above cells on a kernal restart
#cleanedData.to_csv('cleaned_data.csv', index=False)